# Applying Indicators to Price Data
Takes the output from `iq_feed_cleaning.ipynb` and for each row, adds indicators. \
This is preferred over calculating the indicators for each timestamp/date, as it saves 
a lot of computations. \
It does require us to do a look-up, but so does the alternative.

In [1]:
import pandas as pd
import pytz
eastern = pytz.timezone('US/Eastern')
from src.config import config
from concurrent.futures import ThreadPoolExecutor
import os
from src.utils.tickers import get_tickers

In [2]:
tickers = get_tickers(config.data.iqfeed.daily.cleaned)

In [5]:
# Bta calculatiopn (use intraday?)
spy = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/SPY_daily.parquet")
prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/AAPL_daily.parquet")
pd.merge(prices, spy, left_index=True, right_index=True)

In [24]:
def add_indicators(ticker):
    prices = pd.read_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")
    prices["std_252"] = prices["adj_close"].pct_change().rolling(252, min_periods=252).std()*(252**0.5)
    prices["dollar_volume"] = prices["adj_volume"] * (prices["adj_close"] + prices["adj_open"])/2
    prices["r_intra_(t-1)"] = (prices["adj_close"] / prices["adj_open"] - 1).shift(periods=1)
    prices["unadj_open"] = prices["adj_open"] / prices["cum_split_ratio"]
    prices.to_parquet(path=f"{config.data.iqfeed.daily.cleaned}/{ticker}_daily.parquet")

In [25]:
pool_obj = ThreadPoolExecutor(max_workers=os.cpu_count()-1)
ans = pool_obj.map(add_indicators, tickers)
result = list(ans)